### Folder Structure Creation

In [2]:
# Create basic structure
import os
from pathlib import Path

def create_initial_structure():
    base_dir = Path("/home/steve/Python/Emerging-Technologies-in-CpE/facial_recognition_project").expanduser()
    
    directories = [
        'dataset_raw',           # Your original images
        'dataset_processed/original_annotated',  # Standardized images + annotations
        'dataset_processed/augmented',           # Augmented images with auto annotations
        'dataset_processed/train',
        'dataset_processed/val', 
        'models',
        'annotations'            # LabelImg XML files
    ]
    
    for directory in directories:
        (base_dir / directory).mkdir(parents=True, exist_ok=True)
    
    return base_dir

base_dir = create_initial_structure()

### Standardize Original Images

In [9]:
import cv2
class DatasetPreparer:
    def __init__(self, raw_path, output_path):
        self.raw_path = Path(raw_path)
        self.output_path = Path(output_path)
    
    def standardize_for_annotation(self):
        """Prepare standardized images for LabelImg annotation"""
        print("=== Preparing Images for Annotation ===")
        
        person_folders = [f for f in self.raw_path.iterdir() if f.is_dir()]
        
        for person_folder in person_folders:
            person_name = person_folder.name
            output_person_dir = self.output_path / 'original_annotated' / person_name
            output_person_dir.mkdir(parents=True, exist_ok=True)
            
            image_files = list(person_folder.glob('*.*'))
            valid_images = [f for f in image_files if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
            
            print(f"Standardizing {person_name}: {len(valid_images)} images")
            
            for i, img_path in enumerate(valid_images, 1):
                try:
                    # Read and resize image
                    image = cv2.imread(str(img_path))
                    if image is None:
                        continue
                    
                    # Standardize size
                    resized = cv2.resize(image, (640, 640))
                    
                    # Save with standardized name
                    new_filename = f"{person_name}_{i:02d}.jpg"
                    output_path = output_person_dir / new_filename
                    cv2.imwrite(str(output_path), resized)
                    
                    print(f"  Prepared: {img_path.name} -> {new_filename}")
                    
                except Exception as e:
                    print(f"  Error: {img_path.name} -> {e}")

# Run standardization
preparer = DatasetPreparer(
    base_dir / "dataset_raw",
    base_dir / "dataset_processed"
)
preparer.standardize_for_annotation()

=== Preparing Images for Annotation ===
Standardizing Kristina: 5 images
  Prepared: Kristina_02.jpg -> Kristina_01.jpg
  Prepared: Kristina_01.jpg -> Kristina_02.jpg
  Prepared: Kristina_03.jpg -> Kristina_03.jpg
  Prepared: Kristina_04.jpg -> Kristina_04.jpg
  Prepared: Kristina_05.jpg -> Kristina_05.jpg
Standardizing Tonyboy: 5 images
  Prepared: Tonyboy_02.jpg -> Tonyboy_01.jpg
  Prepared: Tonyboy_03.jpg -> Tonyboy_02.jpg
  Prepared: Tonyboy_01.jpg -> Tonyboy_03.jpg
  Prepared: Tonyboy_04.jpg -> Tonyboy_04.jpg
  Prepared: Tonyboy_05.jpg -> Tonyboy_05.jpg
Standardizing Dave: 5 images
  Prepared: Dave_05.jpg -> Dave_01.jpg
  Prepared: Dave_02.jpg -> Dave_02.jpg
  Prepared: Dave_04.jpg -> Dave_03.jpg
  Prepared: Dave_03.jpg -> Dave_04.jpg
  Prepared: Dave_01.jpg -> Dave_05.jpg
Standardizing Cyril: 5 images
  Prepared: Cyril_05.jpg -> Cyril_01.jpg
  Prepared: Cyril_02.jpg -> Cyril_02.jpg
  Prepared: Cyril_03.jpg -> Cyril_03.jpg
  Prepared: Cyril_04.jpg -> Cyril_04.jpg
  Prepared: Cyril

### Partial Standardization

In [30]:
import cv2
from pathlib import Path

class DatasetPreparer:
    def __init__(self, raw_path, output_path):
        self.raw_path = Path(raw_path)
        self.output_path = Path(output_path)
    
    def standardize_single_image(self, image_name, person_name):
        """Standardize a single image and maintain folder structure"""
        print(f"=== Preparing {image_name} for {person_name} ===")
        
        # Input path
        input_path = self.raw_path / person_name / image_name
        
        if not input_path.exists():
            print(f"Error: Image not found at {input_path}")
            return False
        
        # Output path
        output_person_dir = self.output_path / 'augmented' / person_name
        output_person_dir.mkdir(parents=True, exist_ok=True)
        
        try:
            # Read and resize image
            image = cv2.imread(str(input_path))
            if image is None:
                print(f"Error: Could not read image {image_name}")
                return False
            
            # Standardize size
            resized = cv2.resize(image, (640, 640))
            
            # Save with standardized name
            original_stem = Path(image_name).stem
            new_filename = f"{person_name}_{original_stem}.jpg"
            output_path = output_person_dir / new_filename
            cv2.imwrite(str(output_path), resized)
            
            print(f"Success: {image_name} -> {new_filename}")
            return True
            
        except Exception as e:
            print(f"Error processing {image_name}: {e}")
            return False

# Simple usage for your specific case:
preparer = DatasetPreparer(
    base_dir / "dataset_processed/augmented",
    base_dir / "dataset_processed/augemented"
)


# Process multiple specific images
images_to_process = [
    ("Dave_03.jpg", "Dave"),
    ("Tonyboy_01.jpg", "Tonyboy")
]

for image_name, person_name in images_to_process:
    preparer.standardize_single_image(image_name, person_name)

=== Preparing Dave_03.jpg for Dave ===
Success: Dave_03.jpg -> Dave_Dave_03.jpg
=== Preparing Tonyboy_01.jpg for Tonyboy ===
Success: Tonyboy_01.jpg -> Tonyboy_Tonyboy_01.jpg


### Manual Annotation with LabelImg

In [9]:
# After running the standardization above, manually run LabelImg:
print("""
🚀 NOW RUN THESE COMMANDS IN TERMINAL:

1. Activate your environment (if using conda/venv)
2. Run: labelImg

3. In LabelImg:
   - Open Dir: ~/Py/Emerging-Technologies-in-CpE/f/dataset_processed/original_annotated/
   - Save Dir: ~/Py/Emerging-Technologies-in-CpE/f/annotations/
   - Format: PascalVOC (XML)
   - Annotate ALL faces in all images

4. Press Enter here when annotation is complete...
""")


🚀 NOW RUN THESE COMMANDS IN TERMINAL:

1. Activate your environment (if using conda/venv)
2. Run: labelImg

3. In LabelImg:
   - Open Dir: ~/Py/Emerging-Technologies-in-CpE/f/dataset_processed/original_annotated/
   - Save Dir: ~/Py/Emerging-Technologies-in-CpE/f/annotations/
   - Format: PascalVOC (XML)
   - Annotate ALL faces in all images

4. Press Enter here when annotation is complete...



### Dataset Augementation

In [11]:
import os
import cv2
import numpy as np
import json
import xml.etree.ElementTree as ET
from xml.dom import minidom
from pathlib import Path
import random
import math

class SimpleAugmentation:
    def __init__(self):
        self.augmentation_count = 10
    
    def horizontal_flip(self, image, bboxes):
        """Horizontal flip augmentation"""
        flipped_image = cv2.flip(image, 1)
        h, w = image.shape[:2]
        
        flipped_bboxes = []
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            flipped_bbox = [w - x2, y1, w - x1, y2]
            flipped_bboxes.append(flipped_bbox)
        
        return flipped_image, flipped_bboxes
    
    def random_rotation(self, image, bboxes, angle_range=(-15, 15)):
        """Random rotation augmentation"""
        angle = random.uniform(angle_range[0], angle_range[1])
        h, w = image.shape[:2]
        
        # Rotation matrix
        center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        # Rotate image
        rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
        
        # Rotate bounding boxes
        rotated_bboxes = []
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            
            # Transform all four corners
            corners = np.array([
                [x1, y1, 1],
                [x2, y1, 1],
                [x2, y2, 1],
                [x1, y2, 1]
            ])
            
            transformed_corners = np.dot(corners, rotation_matrix.T)
            new_x1 = int(transformed_corners[:, 0].min())
            new_y1 = int(transformed_corners[:, 1].min())
            new_x2 = int(transformed_corners[:, 0].max())
            new_y2 = int(transformed_corners[:, 1].max())
            
            # Ensure within image bounds
            new_x1 = max(0, new_x1)
            new_y1 = max(0, new_y1)
            new_x2 = min(w, new_x2)
            new_y2 = min(h, new_y2)
            
            if new_x2 > new_x1 and new_y2 > new_y1:  # Valid bbox
                rotated_bboxes.append([new_x1, new_y1, new_x2, new_y2])
        
        return rotated_image, rotated_bboxes
    
    def random_brightness_contrast(self, image, bboxes):
        """Random brightness and contrast adjustment"""
        # Brightness adjustment
        brightness = random.uniform(0.7, 1.3)
        # Contrast adjustment
        contrast = random.uniform(0.7, 1.3)
        
        # Apply brightness and contrast
        adjusted_image = cv2.convertScaleAbs(image, alpha=contrast, beta=(brightness - 1) * 128)
        
        return adjusted_image, bboxes  # Bboxes remain the same
    
    def random_scale(self, image, bboxes, scale_range=(0.8, 1.2)):
        """Random scaling augmentation"""
        scale = random.uniform(scale_range[0], scale_range[1])
        h, w = image.shape[:2]
        
        new_w, new_h = int(w * scale), int(h * scale)
        
        # Resize image
        scaled_image = cv2.resize(image, (new_w, new_h))
        
        # Pad or crop to original size
        if scale < 1.0:
            # Pad to original size
            pad_x = (w - new_w) // 2
            pad_y = (h - new_h) // 2
            padded_image = np.zeros((h, w, 3), dtype=np.uint8)
            padded_image[pad_y:pad_y+new_h, pad_x:pad_x+new_w] = scaled_image
            final_image = padded_image
            
            # Adjust bboxes
            scaled_bboxes = []
            for bbox in bboxes:
                x1, y1, x2, y2 = bbox
                new_x1 = int(x1 * scale) + pad_x
                new_y1 = int(y1 * scale) + pad_y
                new_x2 = int(x2 * scale) + pad_x
                new_y2 = int(y2 * scale) + pad_y
                scaled_bboxes.append([new_x1, new_y1, new_x2, new_y2])
                
        else:
            # Crop to original size
            start_x = (new_w - w) // 2
            start_y = (new_h - h) // 2
            final_image = scaled_image[start_y:start_y+h, start_x:start_x+w]
            
            # Adjust bboxes
            scaled_bboxes = []
            for bbox in bboxes:
                x1, y1, x2, y2 = bbox
                new_x1 = max(0, int(x1 * scale) - start_x)
                new_y1 = max(0, int(y1 * scale) - start_y)
                new_x2 = min(w, int(x2 * scale) - start_x)
                new_y2 = min(h, int(y2 * scale) - start_y)
                if new_x2 > new_x1 and new_y2 > new_y1:  # Valid bbox
                    scaled_bboxes.append([new_x1, new_y1, new_x2, new_y2])
        
        return final_image, scaled_bboxes
    
    def add_gaussian_noise(self, image, bboxes):
        """Add Gaussian noise to image"""
        noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
        noisy_image = cv2.add(image, noise)
        return noisy_image, bboxes
    
    def apply_augmentation(self, image, bboxes, aug_type):
        """Apply specific augmentation type"""
        if aug_type == 'flip':
            return self.horizontal_flip(image, bboxes)
        elif aug_type == 'rotation':
            return self.random_rotation(image, bboxes)
        elif aug_type == 'brightness':
            return self.random_brightness_contrast(image, bboxes)
        elif aug_type == 'scale':
            return self.random_scale(image, bboxes)
        elif aug_type == 'noise':
            return self.add_gaussian_noise(image, bboxes)
        else:
            return image, bboxes

class AugmentationWithAnnotations:
    def __init__(self, annotated_images_path, annotations_path, output_path):
        self.annotated_images_path = Path(annotated_images_path)
        self.annotations_path = Path(annotations_path)
        self.output_path = Path(output_path)
        self.augmentation_count = 10
        self.augmentor = SimpleAugmentation()
        
    def load_annotations(self):
        """Load all XML annotations into a dictionary"""
        annotations = {}
        
        for xml_file in self.annotations_path.glob('*.xml'):
            try:
                tree = ET.parse(xml_file)
                root = tree.getroot()
                
                filename = root.find('filename').text
                folder_elem = root.find('folder')
                folder = folder_elem.text if folder_elem is not None else ""
                
                # Store bounding boxes
                bboxes = []
                for obj in root.findall('object'):
                    label = obj.find('name').text
                    bndbox = obj.find('bndbox')
                    xmin = int(float(bndbox.find('xmin').text))
                    ymin = int(float(bndbox.find('ymin').text))
                    xmax = int(float(bndbox.find('xmax').text))
                    ymax = int(float(bndbox.find('ymax').text))
                    
                    bboxes.append({
                        'label': label,
                        'bbox': [xmin, ymin, xmax, ymax]
                    })
                
                # Use filename as key
                annotations[filename] = bboxes
                
            except Exception as e:
                print(f"Error parsing {xml_file}: {e}")
        
        print(f"Loaded annotations for {len(annotations)} images")
        return annotations
    
    def augment_with_annotations(self):
        """Apply augmentation and propagate annotations"""
        print("=== Augmenting Images with Annotation Propagation ===")
        
        annotations = self.load_annotations()
        
        # Define augmentation types
        augmentation_types = ['flip', 'rotation', 'brightness', 'scale', 'noise']
        
        person_folders = [f for f in self.annotated_images_path.iterdir() if f.is_dir()]
        
        total_original = 0
        total_augmented = 0
        
        for person_folder in person_folders:
            person_name = person_folder.name
            output_person_dir = self.output_path / 'augmented' / person_name
            output_person_dir.mkdir(parents=True, exist_ok=True)
            
            image_files = list(person_folder.glob('*.jpg'))
            total_original += len(image_files)
            
            print(f"Augmenting {person_name}: {len(image_files)} images")
            
            for img_path in image_files:
                # Load image
                image = cv2.imread(str(img_path))
                if image is None:
                    print(f"  Could not read: {img_path}")
                    continue
                
                # Get annotations for this image
                if img_path.name not in annotations:
                    print(f"  No annotations found for {img_path.name}")
                    continue
                
                bboxes_data = annotations[img_path.name]
                bboxes = [data['bbox'] for data in bboxes_data]
                labels = [data['label'] for data in bboxes_data]
                
                # Save original with annotations
                original_output_path = output_person_dir / f"{img_path.stem}_original.jpg"
                cv2.imwrite(str(original_output_path), image)
                self.save_annotation(original_output_path, bboxes_data, person_name)
                total_augmented += 1
                
                # Create augmented versions using different augmentation types
                aug_types_used = random.sample(augmentation_types * 2, self.augmentation_count)
                
                for aug_idx, aug_type in enumerate(aug_types_used):
                    try:
                        # Apply augmentation
                        augmented_image, augmented_bboxes = self.augmentor.apply_augmentation(
                            image.copy(), bboxes, aug_type
                        )
                        
                        # Prepare augmented annotation data
                        aug_annotation_data = []
                        for bbox, label in zip(augmented_bboxes, labels):
                            # Ensure bbox coordinates are within image bounds
                            xmin = max(0, int(bbox[0]))
                            ymin = max(0, int(bbox[1]))
                            xmax = min(640, int(bbox[2]))  # 640 is our standardized size
                            ymax = min(640, int(bbox[3]))
                            
                            if xmin < xmax and ymin < ymax:  # Valid bbox
                                aug_annotation_data.append({
                                    'label': label,
                                    'bbox': [xmin, ymin, xmax, ymax]
                                })
                        
                        # Only save if we have valid annotations
                        if aug_annotation_data:
                            # Save augmented image
                            aug_output_path = output_person_dir / f"{img_path.stem}_{aug_type}_{aug_idx+1:02d}.jpg"
                            cv2.imwrite(str(aug_output_path), augmented_image)
                            
                            # Save augmented annotation
                            self.save_annotation(aug_output_path, aug_annotation_data, person_name)
                            total_augmented += 1
                            
                    except Exception as e:
                        print(f"  Error augmenting {img_path} with {aug_type}: {e}")
            
            print(f"  Completed {person_name}")
        
        print(f"\n✅ Augmentation completed!")
        print(f"Original images: {total_original}")
        print(f"Total images after augmentation: {total_augmented}")
        print(f"Multiplication factor: {total_augmented/total_original:.1f}x")
    
    def save_annotation(self, image_path, bboxes_data, folder_name):
        """Save annotation as XML file"""
        image_path = Path(image_path)
        annotation_path = self.annotations_path / f"{image_path.stem}.xml"
        
        # Create XML structure
        annotation = ET.Element('annotation')
        
        # Add folder and filename
        folder_elem = ET.SubElement(annotation, 'folder')
        folder_elem.text = folder_name
        
        filename_elem = ET.SubElement(annotation, 'filename')
        filename_elem.text = image_path.name
        
        # Add size (assuming 640x640 from standardization)
        size_elem = ET.SubElement(annotation, 'size')
        ET.SubElement(size_elem, 'width').text = '640'
        ET.SubElement(size_elem, 'height').text = '640'
        ET.SubElement(size_elem, 'depth').text = '3'
        
        # Add each object/bbox
        for bbox_data in bboxes_data:
            obj_elem = ET.SubElement(annotation, 'object')
            ET.SubElement(obj_elem, 'name').text = bbox_data['label']
            ET.SubElement(obj_elem, 'pose').text = 'Unspecified'
            ET.SubElement(obj_elem, 'truncated').text = '0'
            ET.SubElement(obj_elem, 'difficult').text = '0'
            
            bndbox_elem = ET.SubElement(obj_elem, 'bndbox')
            xmin, ymin, xmax, ymax = bbox_data['bbox']
            ET.SubElement(bndbox_elem, 'xmin').text = str(xmin)
            ET.SubElement(bndbox_elem, 'ymin').text = str(ymin)
            ET.SubElement(bndbox_elem, 'xmax').text = str(xmax)
            ET.SubElement(bndbox_elem, 'ymax').text = str(ymax)
        
        # Save XML file
        xml_str = minidom.parseString(ET.tostring(annotation)).toprettyxml(indent="  ")
        with open(annotation_path, 'w') as f:
            f.write(xml_str)

# Install only what we need
def install_requirements():
    """Install required packages without PyTorch dependencies"""
    import subprocess
    import sys
    
    packages = [
        "opencv-python",
        "numpy", 
        "Pillow",
        "insightface",
        "scikit-learn"
    ]
    
    for package in packages:
        try:
            __import__(package.replace("-", "_"))
            print(f"✅ {package} already installed")
        except ImportError:
            print(f"📦 Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install requirements
print("=== Installing/Checking Requirements ===")
install_requirements()

# Now run the augmentation
print("\n=== Starting Augmentation with Simple Augmentations ===")

# Define paths
base_dir = Path("/home/steve/Python/Emerging-Technologies-in-CpE/facial_recognition_project").expanduser()

# Run augmentation with annotation propagation
augmenter = AugmentationWithAnnotations(
    base_dir / "dataset_processed" / "original_annotated",
    base_dir / "annotations", 
    base_dir / "dataset_processed"
)
augmenter.augment_with_annotations()

=== Installing/Checking Requirements ===
📦 Installing opencv-python...



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /home/steve/venvs/facerec/bin/python -m pip install --upgrade pip


✅ numpy already installed
📦 Installing Pillow...



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /home/steve/venvs/facerec/bin/python -m pip install --upgrade pip


✅ insightface already installed
📦 Installing scikit-learn...



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /home/steve/venvs/facerec/bin/python -m pip install --upgrade pip



=== Starting Augmentation with Simple Augmentations ===
=== Augmenting Images with Annotation Propagation ===
Loaded annotations for 50 images
Augmenting Kristina: 5 images
  Completed Kristina
Augmenting Tonyboy: 5 images
  Completed Tonyboy
Augmenting Dave: 5 images
  Completed Dave
Augmenting Cyril: 5 images
  Completed Cyril
Augmenting .comments: 0 images
  Completed .comments
Augmenting Mars: 5 images
  Completed Mars
Augmenting Steve: 5 images
  Completed Steve
Augmenting Lovely: 5 images
  Completed Lovely
Augmenting Sheryl: 5 images
  Completed Sheryl
Augmenting Laurentti: 5 images
  Completed Laurentti
Augmenting Danica: 5 images
  Completed Danica

✅ Augmentation completed!
Original images: 50
Total images after augmentation: 550
Multiplication factor: 11.0x


### Convert all annotations to JSON

In [31]:
import json
import xml.etree.ElementTree as ET
from pathlib import Path

class AnnotationConverter:
    def __init__(self, annotations_path, output_path):
        self.annotations_path = Path(annotations_path)
        self.output_path = Path(output_path)
        
    def convert_all_annotations_to_json(self):
        """Convert all XML annotations to a single JSON file for InsightFace"""
        all_annotations = {}
        
        # Get all XML files (original + augmented)
        xml_files = list(self.annotations_path.glob('*.xml'))
        print(f"Found {len(xml_files)} annotation files to convert")
        
        for xml_file in xml_files:
            try:
                tree = ET.parse(xml_file)
                root = tree.getroot()
                
                filename = root.find('filename').text
                folder_elem = root.find('folder')
                folder = folder_elem.text if folder_elem is not None else ""
                
                # Construct image path
                if folder:
                    image_path = f"dataset_processed/augmented/{folder}/{filename}"
                else:
                    # Try to find which folder this image belongs to
                    image_path = self.find_image_path(filename)
                
                # Extract bounding boxes
                bboxes = []
                for obj in root.findall('object'):
                    label = obj.find('name').text
                    bndbox = obj.find('bndbox')
                    xmin = int(float(bndbox.find('xmin').text))
                    ymin = int(float(bndbox.find('ymin').text))
                    xmax = int(float(bndbox.find('xmax').text))
                    ymax = int(float(bndbox.find('ymax').text))
                    
                    bboxes.append({
                        'label': label,
                        'bbox': [xmin, ymin, xmax, ymax]
                    })
                
                if bboxes:  # Only add if we have annotations
                    all_annotations[image_path] = bboxes
                    
            except Exception as e:
                print(f"Error converting {xml_file}: {e}")
        
        # Save as JSON
        output_file = self.output_path / "annotations.json"
        with open(output_file, 'w') as f:
            json.dump(all_annotations, f, indent=2)
        
        print(f"✅ Converted {len(all_annotations)} annotations to {output_file}")
        return all_annotations
    
    def find_image_path(self, filename):
        """Find which folder contains this image"""
        augmented_path = base_dir / "dataset_processed" / "augmented"
        for person_folder in augmented_path.iterdir():
            if person_folder.is_dir():
                image_path = person_folder / filename
                if image_path.exists():
                    return f"dataset_processed/augmented/{person_folder.name}/{filename}"
        return f"dataset_processed/augmented/unknown/{filename}"

# Convert annotations to JSON
print("=== STEP 5: Converting Annotations to JSON ===")
converter = AnnotationConverter(
    base_dir / "annotations",
    base_dir / "dataset_processed"
)
annotations_json = converter.convert_all_annotations_to_json()

=== STEP 5: Converting Annotations to JSON ===
Found 50 annotation files to convert
✅ Converted 50 annotations to /home/steve/Python/Emerging-Technologies-in-CpE/facial_recognition_project/dataset_processed/annotations.json


### Train InsightFace model

In [32]:
from insightface.app import FaceAnalysis
import pickle

class InsightFaceTrainer:
    def __init__(self, model_name='buffalo_l'):
        self.model = FaceAnalysis(name=model_name, providers=['CPUExecutionProvider'])
        self.model.prepare(ctx_id=0, det_size=(640, 640))
        self.face_database = {}  # {person_id: [embedding1, embedding2, ...]}
        self.threshold = 0.65  # Similarity threshold
    
    def build_face_database(self, annotations_file):
        """Build face database from annotations - WITH FALLBACK DETECTION"""
        print("=== Building Face Database ===")
        
        with open(annotations_file, 'r') as f:
            annotations = json.load(f)
        
        processed_count = 0
        skipped_count = 0
        
        for image_path, objects in annotations.items():
            full_image_path = base_dir / image_path
            
            if not full_image_path.exists():
                print(f"Image not found: {full_image_path}")
                skipped_count += 1
                continue
            
            for obj in objects:
                person_id = obj['label']
                bbox = obj['bbox']
                
                # Try multiple methods to extract face embedding
                embedding = self.extract_face_embedding_robust(str(full_image_path), bbox)
                
                if embedding is not None:
                    if person_id not in self.face_database:
                        self.face_database[person_id] = []
                    
                    self.face_database[person_id].append(embedding)
                    processed_count += 1
                    print(f"✅ Added embedding for {person_id} from {Path(image_path).name}")
                else:
                    skipped_count += 1
                    print(f"❌ Failed to extract embedding from {Path(image_path).name}")
        
        print(f"\nDatabase built successfully!")
        print(f"Processed: {processed_count} faces")
        print(f"Skipped: {skipped_count} faces")
        print(f"Total persons: {len(self.face_database)}")
        
        # Print person-wise counts
        for person, embeddings in self.face_database.items():
            print(f"  {person}: {len(embeddings)} embeddings")
    
    def extract_face_embedding_robust(self, image_path, bbox):
        """Robust face extraction with multiple fallback methods"""
        try:
            image = cv2.imread(image_path)
            if image is None:
                return None
            
            # METHOD 1: Try with original bounding box
            embedding = self._extract_with_bbox(image, bbox)
            if embedding is not None:
                return embedding
            
            # METHOD 2: Try with expanded bounding box (if bbox was too tight)
            embedding = self._extract_with_expanded_bbox(image, bbox)
            if embedding is not None:
                return embedding
            
            # METHOD 3: Let InsightFace auto-detect face in the entire image
            embedding = self._extract_with_auto_detect(image)
            if embedding is not None:
                return embedding
            
            return None
                
        except Exception as e:
            print(f"Error extracting embedding from {Path(image_path).name}: {e}")
            return None
    
    def _extract_with_bbox(self, image, bbox):
        """Extract using provided bounding box"""
        x1, y1, x2, y2 = bbox
        
        # Ensure bounding box is within image bounds
        h, w = image.shape[:2]
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x2)
        y2 = min(h, y2)
        
        # Check if bbox is valid
        if x2 <= x1 or y2 <= y1:
            return None
        
        # Crop face region
        face_crop = image[y1:y2, x1:x2]
        
        if face_crop.size == 0:
            return None
        
        # Ensure minimum size for face detection
        if face_crop.shape[0] < 20 or face_crop.shape[1] < 20:
            return None
        
        # Get face embedding from cropped region
        faces = self.model.get(face_crop)
        if len(faces) == 1 and hasattr(faces[0], 'embedding'):
            return faces[0].embedding
        
        return None
    
    def _extract_with_expanded_bbox(self, image, bbox, expansion_factor=0.2):
        """Extract using expanded bounding box"""
        x1, y1, x2, y2 = bbox
        h, w = image.shape[:2]
        
        # Expand bounding box
        width = x2 - x1
        height = y2 - y1
        expand_x = int(width * expansion_factor)
        expand_y = int(height * expansion_factor)
        
        new_x1 = max(0, x1 - expand_x)
        new_y1 = max(0, y1 - expand_y)
        new_x2 = min(w, x2 + expand_x)
        new_y2 = min(h, y2 + expand_y)
        
        return self._extract_with_bbox(image, [new_x1, new_y1, new_x2, new_y2])
    
    def _extract_with_auto_detect(self, image):
        """Let InsightFace auto-detect face in entire image"""
        faces = self.model.get(image)
        
        # Use the largest face if multiple detected
        if len(faces) >= 1:
            # Find the largest face by bounding box area
            largest_face = max(faces, key=lambda face: 
                             (face.bbox[2] - face.bbox[0]) * (face.bbox[3] - face.bbox[1]))
            
            if hasattr(largest_face, 'embedding'):
                return largest_face.embedding
        
        return None
    
    def save_database(self, output_path):
        """Save face database to file"""
        # Create directory if it doesn't exist
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(output_path, 'wb') as f:
            pickle.dump(self.face_database, f)
        print(f"✅ Face database saved to {output_path}")
    
    def load_database(self, input_path):
        """Load face database from file"""
        with open(input_path, 'rb') as f:
            self.face_database = pickle.load(f)
        print(f"✅ Face database loaded from {input_path}")

# Train the InsightFace model
print("\n=== STEP 6: Training InsightFace Model ===")
trainer = InsightFaceTrainer()

# Build database from annotations
annotations_file = base_dir / "dataset_processed" / "annotations.json"

# Check if annotations file exists
if not annotations_file.exists():
    print(f"❌ Annotations file not found: {annotations_file}")
    print("Please run the annotation conversion step first!")
else:
    print(f"✅ Found annotations file: {annotations_file}")
    trainer.build_face_database(annotations_file)
    
    # Save the trained model
    model_path = base_dir / "models" / "face_database.pkl"
    trainer.save_database(model_path)
    
    # Verify the database was created
    if trainer.face_database:
        print(f"\n🎉 Training completed successfully!")
        print(f"   Persons in database: {len(trainer.face_database)}")
        total_embeddings = sum(len(embeddings) for embeddings in trainer.face_database.values())
        print(f"   Total face embeddings: {total_embeddings}")
    else:
        print(f"\n❌ Training failed - no face embeddings were extracted!")


=== STEP 6: Training InsightFace Model ===
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/w600k_r50.onnx recogniti

### Run the Facial Recognition Display

In [2]:
import subprocess
import os

env = os.environ.copy()
env['QT_QPA_PLATFORM'] = 'xcb'
subprocess.Popen(["/home/steve/venvs/facerec/bin/python", "interactive.py"], env=env)

<Popen: returncode: None args: ['/home/steve/venvs/facerec/bin/python', 'int...>

🚀 LAUNCHING FACE RECOGNITION SYSTEM - RESIZABLE WINDOWS
📦 Step 1: Loading trained model...
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insigh